In [32]:
from sqlalchemy import create_engine, MetaData

# 创建数据库引擎
engine = create_engine('sqlite:///example.db', echo=True)
# 创建元数据对象，用于保存表定义
metadata =MetaData()

In [ ]:
from sqlalchemy import Table, Column, Integer, String, Sequence

# 定义表结构
# 表名为 user， metadata 是一个包含表定义的容器，管理着表和数据库模式的集合。
user_table = Table('user', metadata,
                   # Column 定义了表中的一列，id 是列的名称。
                   # Integer 指定该列的数据类型为整数。 
                   # Sequence('user_id_seq') 指定一个序列，用于自动生成唯一的ID值。
                   # primary_key=True 指定该列是主键，表中每行的唯一标识符。
                   Column('id', Integer, Sequence('user_id_seq'), primary_key=True),
                   Column('name', String(50)),
                   Column('age', Integer))
# 创建所有由 metadata 定义的表，这里是 user_table
metadata.create_all(engine)

In [ ]:
from sqlalchemy import insert

# engine.connect()创建一个数据库连接，并在 with 区块结束时自动关闭连接
with engine.connect() as connection:
    # 插入数据，指定目标表为 user_table，.values设置要插入的行数据，指定 name 列的值为 John Doe，age 列的值为 100
    stmt = insert(user_table).values(name='John Doe', age=100)
    # 执行插入操作 stmt ，将数据插入到数据库中
    result = connection.execute(stmt)
    print('插入的行ID: ', result.inserted_primary_key)

In [ ]:
from sqlalchemy import select

# 查询数据
with engine.connect() as connection:
    stmt = select(user_table)
    result = connection.execute(stmt)
    for row in result:
        print('ID: ', row['id'], 'Name: ', row['name'], 'Age: ', row['age'])


In [ ]:
from sqlalchemy import update

# 更新数据
with engine.connect() as connection:
    stmt = update(user_table).where(user_table.c.name == 'John Doe').values(age=30)
    result = connection.execute(stmt)
    print('更新的行数: ', result.rowcount)

In [ ]:
from sqlalchemy import delete

# 删除数据
with engine.connect() as connection:
    stmt = delete(user_table).where(user_table.c.name == 'John Doe')
    result = connection.execute(stmt)
    print('删除的行数: ', result.rowcount)

In [ ]:
# 使用 ORM 映射类
from sqlalchemy.orm import sessionmaker, declarative_base

# 创建基类
Base = declarative_base()

# 定义映射类
class User(Base):
    __tablename__ = 'user'
    
    id = Column(Integer, Sequence('user_id_seq'), primary_key=True)
    name = Column(String(50))
    age = Column(Integer)
    
    def __repr__(self):
        return f'<User(name={self.name}, age={self.age})>'

# 创建所有表
Base.metadata.create_all(engine)

# 创建会话
Session = sessionmaker(bind=engine)
session = Session()

# 插入数据
new_user = User(name='John', age=30)
session.add(new_user)
session.commit()

# 查询数据
users = session.query(User).all()
for user in users:
    print(user.id, user.name, user.age)

# 更新数据
user = session.query(User).filter_by(name='John').first()
user.age = 100
session.commit()

# 删除数据
user = session.query(User).filter_by(name='John').first()
session.delete(user)
session.commit()
